In [34]:
import pandas as pd

In [35]:
FILE = "wineQualityReds.csv"
df = pd.read_csv(FILE)
df = df.drop(["Unnamed: 0"], axis="columns")
df

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [36]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# функция создания пайплайна для модели (со стандартным масштабированием) и проверки точности модели
# fit на тренировочной и score на тестовой выборках
def check_model(model, X_train, y_train, X_test, y_test):
    clf = make_pipeline(StandardScaler(), model)
    clf.fit(X_train, y_train)
    return clf.score(X_test, y_test)


In [37]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
RANDOM_STATE = 0
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(["quality"], axis="columns"), df["quality"],
    test_size=TEST_SIZE, random_state=RANDOM_STATE)
X_train

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol
642,9.9,0.540,0.45,2.3,0.071,16.0,40.0,0.99910,3.39,0.62,9.4
679,10.8,0.260,0.45,3.3,0.060,20.0,49.0,0.99720,3.13,0.54,9.6
473,9.9,0.350,0.55,2.1,0.062,5.0,14.0,0.99710,3.26,0.79,10.6
390,5.6,0.850,0.05,1.4,0.045,12.0,88.0,0.99240,3.56,0.82,12.9
1096,6.6,0.725,0.09,5.5,0.117,9.0,17.0,0.99655,3.35,0.49,10.8
...,...,...,...,...,...,...,...,...,...,...,...
763,9.3,0.655,0.26,2.0,0.096,5.0,35.0,0.99738,3.25,0.42,9.6
835,7.6,0.665,0.10,1.5,0.066,27.0,55.0,0.99655,3.39,0.51,9.3
1216,7.9,0.570,0.31,2.0,0.079,10.0,79.0,0.99677,3.29,0.69,9.5
559,13.0,0.470,0.49,4.3,0.085,6.0,47.0,1.00210,3.30,0.68,12.7


In [38]:
# Алгоритм SVM ищет объекты данных (вектора), которые расположены ближе
# всего к линии разделения. Эти точки называются опорными векторами. 
# Затем, алгоритм вычисляет расстояние между опорными векторами и 
# разделяющей плоскостью (это расстояние называется зазором). Основная
# цель алгоритма — максимизировать расстояние зазора. Лучшей
# гиперплоскостью считается такая гиперплоскость, для которой этот
# зазор является максимально большим.

from sklearn.svm import SVC, LinearSVC
svc_model = SVC(gamma='auto')
acc_svc = check_model(svc_model, X_train, y_train, X_test, y_test)
linear_svc_model = LinearSVC(random_state=0, tol=1e-5, max_iter=1e6)
acc_linear_svc = check_model(linear_svc_model, X_train, y_train, X_test, y_test)
acc_svc, acc_linear_svc

(0.64375, 0.634375)

In [39]:
# kNN (k Nearest Neighbor) – алгоритм k-ближайших соседей использует
# весь набор данных в качестве обучающего набора, а не разделяет набор
# данных на обучающий набор и набор тестов. Когда для нового экземпляра
# данных требуется результат, алгоритм KNN просматривает весь набор
# данных, чтобы найти k-ближайших экземпляров для нового экземпляра, 
# или k экземпляров, наиболее похожих на новую запись, а затем выводит
# среднее значение результаты (для регрессии) или наиболее близкий
# класс (для задачи классификации).

from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=50)
acc_knn = check_model(knn_model, X_train, y_train, X_test, y_test)
acc_knn

0.6

In [40]:
# Наи́вный ба́йесовский классифика́тор — простой вероятностный 
# классификатор, основанный на применении теоремы Байеса со строгими
# (наивными) предположениями о независимости.
from sklearn.naive_bayes import GaussianNB

gaussian_model = GaussianNB()
acc_gaussian = check_model(gaussian_model, X_train, y_train, X_test, y_test)
acc_gaussian

0.540625

In [41]:
# Логистическая регрессия — это статистическая модель, используемая
# для прогнозирования вероятности возникновения некоторого события
# путём его сравнения с логистической кривой. Эта регреcсия выдаёт
# ответ в виде вероятности бинарного события (1 или 0).

#  Стохастический градиентный спуск - это метод итерации для
# оптимизации целевой функции с подходящими свойствами гладкости
# (например, дифференцируемость или субдифференцируемость). Его можно
# расценивать как стохастическую аппроксимацию оптимизации методом 
# градиентного спуска, поскольку он заменяет реальный градиент 
# (вычисленный из полного набора данных[en]) путём оценки такового
# (вычисленного из случайно выбранного подмножества данных)

from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier

log_model = LogisticRegression(random_state=0)
acc_log = check_model(log_model, X_train, y_train, X_test, y_test)
perceptron_model = Perceptron(tol=1e-1, random_state=0)
acc_perceptron = check_model(perceptron_model, X_train, y_train, X_test, y_test)
sgd_model = SGDClassifier(max_iter=1e6, tol=1e-3)
acc_sgd = check_model(sgd_model, X_train, y_train, X_test, y_test)
acc_log, acc_perceptron, acc_sgd

(0.634375, 0.540625, 0.5125)

In [42]:
# Деревья решений.
# Идея: Дерево строится «сверху вниз» от корня. 
# Начинается процесс с определения, какой атрибут следует выбрать
# для проверки в корне дерева. Для этого каждый атрибут исследуется
# на предмет, как хорошо он классифицирует набор данных (разделяет
# на классы по целевому атрибуту). При этом выбирается тот из
# атрибутов, который порождает наибольший количественный критерий 
# оценки. Когда атрибут выбран, для каждого его значения создается
# ветка дерева, набор данных разделяется в соответствии со значением
# к каждой ветке, процесс повторяется рекурсивно для каждой ветки.
# Также следует проверять критерий остановки.

# В деревьях классификации часто используются перекрестная энтропия,
# энтропия Шеннона и коэффициент Джини. В деревьях регрессии
# минимизируется сумма функций потерь. Мы выполняем эту процедуру
# рекурсивно для каждого узла и завершаем работу, когда выполняем 
# критерии остановки.

# В качестве критерия остановки могут быть выбраны: минимальное
# количество уровней дерева от листа до вершины, минимальное
# значение критерия оценки в узле и пр.

from sklearn.tree import DecisionTreeClassifier

decision_tree_model = DecisionTreeClassifier(random_state=0)
acc_decision_tree = check_model(decision_tree_model, X_train, y_train, X_test, y_test)
acc_decision_tree

0.6875

In [43]:
# Random forest (с англ. — «случайный лес») — алгоритм машинного 
# обучения, заключающийся в использовании комитета (ансамбля) решающих
# деревьев. Алгоритм сочетает в себе две основные идеи: метод бэггинга
# Бреймана, и метод случайных подпространств, предложенный Тин Кам Хо.
# Алгоритм применяется для задач классификации, регрессии и
# кластеризации. Основная идея заключается в использовании большого 
# ансамбля решающих деревьев, каждое из которых само по себе даёт
# очень невысокое качество классификации, но за счёт их большого
# количества результат получается хорошим.

from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=1000, random_state=0)
acc_random_forest = check_model(random_forest_model, X_train, y_train, X_test, y_test)
acc_random_forest

0.728125

In [44]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression',
              'Random Forest', 'Naive Bayes', 'Perceptron',
              'Stochastic Gradient Decent', 'Linear SVC',
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log,
              acc_random_forest, acc_gaussian, acc_perceptron,
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Object': [svc_model, knn_model, log_model, random_forest_model,
              gaussian_model, perceptron_model, sgd_model,
              linear_svc_model, decision_tree_model]}).sort_values(by='Score', ascending=False)
models

,Model,Score,Object
3,Random Forest,0.728125,"(DecisionTreeClassifier(max_features='auto', r..."
8,Decision Tree,0.687500,DecisionTreeClassifier(random_state=0)
0,Support Vector Machines,0.643750,SVC(gamma='auto')
2,Logistic Regression,0.634375,LogisticRegression(random_state=0)
7,Linear SVC,0.634375,"LinearSVC(max_iter=1000000.0, random_state=0, ..."
1,KNN,0.600000,KNeighborsClassifier(n_neighbors=50)
4,Naive Bayes,0.540625,GaussianNB()
5,Perceptron,0.540625,Perceptron(tol=0.1)
6,Stochastic Gradient Decent,0.512500,SGDClassifier(max_iter=1000000.0)


In [45]:
# Выгрузка модели в файл

import os
import pickle
import shutil

with open("best.pkl", "wb") as f:
    pickle.dump(models['Object'][0], f)

shutil.copy('best.txt', os.path.join('..', '..', 'app', 'backend', 'scripts'))

In [47]:
# Загрузка модели из файла

import pickle

with open("best.pkl", "rb") as f:
    model = pickle.load(f)
model.predict([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1]])

array([5])